# Generating trading signals with Gradient Boosting

This notebook illustrates the following steps:
1. **Cross-validation in the time-series context** poses the additional challenge that train and validation sets need to respect the temporal order of the data so that we do not inadvertently train the model on data 'from the future' to predict the past and introduce lookahead bias. Scikit-learn's built-in [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) aims to accomplish this but does not work for this case where we have multiple time series, one for each ticker. We could solve this by manually subsetting the data for the appropriate train and validation periods. Alternatively, we can create a custom time-series splitter compatible with the scikit-learn Kfold interface (see resources). Here is an [example](https://github.com/stefan-jansen/machine-learning-for-trading/blob/master/utils.py) that illustrates how to do so for this case. This allows us to specify fixed `train_length` and `test_length` parameters, as well as a `lookahead` value that defines the forecast horizon and ensures an appropriate gap between the training and validation set.
2. Now, we'll investigate if a more complex **gradient boosting** model is capable of improving the result. 
    - We'll use the LightGBM implementation because it often [outperforms](https://lightgbm.readthedocs.io/en/latest/Experiments.html) the alternative [XGBoost](https://xgboost.readthedocs.io/en/latest/) and [CatBoost](https://catboost.ai/) libraries, especially in terms of runtime and memory footprint. There is also a recent scikit-learn [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor) inspired by LightGBM. 
    - You need to define and [tune](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) several **hyperparameters** to define the model's complexity as well as constraints on the learning process (see [docs](https://lightgbm.readthedocs.io/en/latest/Parameters.html) for the full list). The goal is to adjust the model's capacity to the data while limiting the risk of overfitting so that the model learns the signal rather than the noise in the data and generalizes better to out-of-sample data. In other words, we aim to identify the most promising model settings based on an unbiased estimate of the generalization error, and then apply these settings to generate out-of-sample predictions.
    - The **most important parameters** define the *number of iterations* or trees, and the *size of each tree*. LightGBM allows you to optimize the number of iterations by training a model for a certain number of iterations, then evaluating its performance before continuing to train the model. LightGBM permits [leaf-wise tree growth](https://lightgbm.readthedocs.io/en/latest/Features.html#leaf-wise-best-first-tree-growth) and let's you limit the size of the tree by setting the number of leaf nodes or by limiting the depth of the tree. A useful combination targets a certain number of leaves while constraining the minimum number of samples per leaf node to avoid excessive tree imbalances as well as noisy estimates due to leaf nodes with few samples. The default value of 32 of the number of leaves is a good starting point, but the min. number of data points per leaves should be significantly higher than the default of 20 - with 1m observations and 100 leave nodes, each leaf would on average contain 10K samples. 
    - A lower learning rate can help boost accuracy but requires more iterations and, thus, extends training time. See also LightGBM's [guidance](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) on parameter tuning.
    - Finally, the parameter `feature_frac` let's you control the amount of randomization while building each tree.
3. To **evaluate the model performance**, compare the cross-validation IC for different hyperparameter settings, computed again as the average of the Spearman rank correlation of the model predictions for each day with the actual returns. Identify the best-performing 5-10 models over a number of recent quarters, and check whether averaging their predictions further improves the result. In addition, use Alphalens to compute a summary tearsheet and inspect the spread between the botton and the top quintile of the predictions.
4. Generate out-of-sample predictions for 2014-2016 using your preferred models (which may change for every quarter), and possibly average the result if you found this to yield more stable results. Repeat your Alphalens analysis for these signals.

## Training with GPU

> If you have a GPU, you can install the LightGBM GPU version instead of the CPU version to get a [decent speedup](https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html). To accomplish this, run
```bash
conda remove -n liveproject lightgbm -y
pip install lightgbm --install-option=--gpu
```
You will also need to adapt the LightGBM parameter settings below to indicate that you'll be using the `device` `GPU`. A smaller bin size like 63 tends to perform better than the default 255.

> You can also upload and run this notebook in [google collab](https://colab.research.google.com/notebooks/intro.ipynb), just install the packages required by the below imports, mount you Google Drive as described [here](https://buomsoo-kim.github.io/colab/2020/05/09/Colab-mounting-google-drive.md/) and update the `DATA_PATH` to the location on your Google Drive (relative to you mount path) with the `HDFStore` containing the Quandl dataset we created in the first milestone.

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

from pathlib import Path
import sys, os
from time import time
from collections import defaultdict
from itertools import product

import numpy as np
import pandas as pd
import statsmodels.api as sm

import lightgbm as lgb

from scipy.stats import spearmanr

from alphalens.tears import (create_summary_tear_sheet,
                             create_full_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

import matplotlib.pyplot as plt
import seaborn as sns

## Settings

In [3]:
sns.set_style('whitegrid')

In [4]:
np.random.seed(42)

In [5]:
idx = pd.IndexSlice

In [6]:
YEAR = 252 # days
MONTH = 21 # days

In [7]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}'

## Get Data

In [8]:
data = (pd.read_hdf('stock_prices.h5', 'model_data')
        .sort_index())

In [9]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1402295 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZMH', Timestamp('2015-07-02 00:00:00'))
Data columns (total 48 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ret_01       1401795 non-null  float64
 1   ret_03       1400795 non-null  float64
 2   ret_05       1399795 non-null  float64
 3   ret_10       1397295 non-null  float64
 4   ret_21       1391795 non-null  float64
 5   ret_42       1381295 non-null  float64
 6   ret_63       1370795 non-null  float64
 7   ret_126      1339295 non-null  float64
 8   ret_252      1276295 non-null  float64
 9   ret_fwd      1402295 non-null  float64
 10  BB_UP        1392795 non-null  float64
 11  BB_LOW       1392795 non-null  float64
 12  BB_SQUEEZE   1392795 non-null  float64
 13  HT           1370795 non-null  float64
 14  SAR          1401795 non-null  float64
 15  ADX          1388795 non-null  float64
 16  ADXR         13822

To facilitate model formulation, we assign target, features, and categorical variables.

In [10]:
target = 'ret_fwd'
features = data.columns.drop(target)

In [11]:
categoricals = ['month', 'weekday']

## Custom Time Series Cross-Validation

See milestone 4 solution.

In [12]:
class MultipleTimeSeriesCV:
    """Generates tuples of train_idx, test_idx pairs
    Assumes the MultiIndex contains levels 'symbol' and 'date'
    purges overlapping outcomes"""

    def __init__(self,
                 n_splits=3,
                 train_period_length=126,
                 test_period_length=21,
                 lookahead=None,
                 date_idx='date',
                 shuffle=False):
        self.n_splits = n_splits
        self.lookahead = lookahead
        self.test_length = test_period_length
        self.train_length = train_period_length
        self.shuffle = shuffle
        self.date_idx = date_idx

    def split(self, X, y=None, groups=None):
        unique_dates = X.index.get_level_values(self.date_idx).unique()
        days = sorted(unique_dates, reverse=True)
        split_idx = []
        for i in range(self.n_splits):
            test_end_idx = i * self.test_length
            test_start_idx = test_end_idx + self.test_length
            train_end_idx = test_start_idx + self.lookahead - 1
            train_start_idx = train_end_idx + self.train_length + self.lookahead - 1
            split_idx.append([train_start_idx, train_end_idx,
                              test_start_idx, test_end_idx])

        dates = X.reset_index()[[self.date_idx]]
        for train_start, train_end, test_start, test_end in split_idx:

            train_idx = dates[(dates[self.date_idx] > days[train_start])
                              & (dates.date <= days[train_end])].index
            test_idx = dates[(dates.date > days[test_start])
                             & (dates.date <= days[test_end])].index
            if self.shuffle:
                np.random.shuffle(list(train_idx))
            yield train_idx.to_numpy(), test_idx.to_numpy()

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits


## Model Selection: Lookback, lookahead and roll-forward periods

We'll use five years of training data to predict the 1-day forward returns for the following three months just as for the linear regression models.

In [13]:
train_length = 5 * YEAR
test_length = 3 * MONTH

In [14]:
lookahead = 1

We'll create 16 quarters worth of out-of-sample predictions so we can select the past 4 quarters to select a model for a subsequent quarter with three years total of out-of-sample forecasts.

In [15]:
n_splits = 16

In [16]:
cv = MultipleTimeSeriesCV(n_splits=n_splits,
                          test_period_length=test_length,
                          lookahead=lookahead,
                          train_period_length=train_length)

Show rolling cross-validation periods - note that the folds move backwards in time:

In [17]:
for n_split, (train_idx, test_idx) in enumerate(cv.split(X=data)):
    train = data.iloc[train_idx]
    train_dates = train.index.get_level_values('date')
    
    test = data.iloc[test_idx]
    test_dates = test.index.get_level_values('date')
    
    train_days = train.groupby(level="ticker").size().value_counts().index[0]
    train_start, train_end = train_dates.min().date(), train_dates.max().date()
    
    test_days = test.groupby(level="ticker").size().value_counts().index[0]
    test_start, test_end = test_dates.min().date(), test_dates.max().date()
                                                  
    print(f'Split: {n_split:02} | # Train: {train_start} - {train_end} ({train_days:5,.0f} days) | '
          f'Test: {test_start} - {test_end} ({test_days} days)')

Split: 00 | # Train: 2011-11-08 - 2016-09-29 (1,260 days) | Test: 2016-09-30 - 2016-12-29 (63 days)
Split: 01 | # Train: 2011-08-11 - 2016-06-30 (1,260 days) | Test: 2016-07-01 - 2016-09-29 (63 days)
Split: 02 | # Train: 2011-05-16 - 2016-04-01 (1,260 days) | Test: 2016-04-04 - 2016-06-30 (63 days)
Split: 03 | # Train: 2011-02-14 - 2015-12-30 (1,260 days) | Test: 2015-12-31 - 2016-04-01 (63 days)
Split: 04 | # Train: 2010-11-17 - 2015-09-30 (1,260 days) | Test: 2015-10-01 - 2015-12-30 (63 days)
Split: 05 | # Train: 2010-08-20 - 2015-07-01 (1,260 days) | Test: 2015-07-02 - 2015-09-30 (63 days)
Split: 06 | # Train: 2010-05-25 - 2015-04-01 (1,260 days) | Test: 2015-04-02 - 2015-07-01 (63 days)
Split: 07 | # Train: 2010-02-25 - 2014-12-30 (1,260 days) | Test: 2014-12-31 - 2015-04-01 (63 days)
Split: 08 | # Train: 2009-11-26 - 2014-09-30 (1,260 days) | Test: 2014-10-01 - 2014-12-30 (63 days)
Split: 09 | # Train: 2009-08-31 - 2014-07-03 (1,260 days) | Test: 2014-07-04 - 2014-09-30 (63 days)


## LightGBM Model Tuning

### Compute Feature Importance

In [18]:
def get_fi(model):
    fi = model.feature_importance(importance_type='gain')
    return (pd.Series(fi / fi.sum(),
                      index=model.feature_name()))

### Hyperparameter Options

See LightGBM [docs](https://lightgbm.readthedocs.io/en/latest/Parameters.html) for details on the various hyperparameters.

In [19]:
base_params = dict(boosting='gbdt',
                   objective='regression',
                   metric='None',
#                    device='gpu', # uncomment when using GPU
#                    max_bin=63,   # uncomment when using GPU
                   verbose=-1)

Here are some reasonable hyperparameter choices; feel free to experiment!

In [20]:
learning_rate_ops = [.01]
max_depths = [5, 6, 7]

# we'll set the num_leaves parameter to enable leaf-wise growth
num_leaves_opts = [2 ** i for i in max_depths]
feature_fraction_opts = [.6, .95]
min_data_in_leaf_opts = [250, 750, 1500]

In [21]:
param_names = ['learning_rate', 
               'num_leaves',
               'feature_fraction', 
               'min_data_in_leaf']

In [22]:
cv_params = list(product(learning_rate_ops,
                         num_leaves_opts,
                         feature_fraction_opts,
                         min_data_in_leaf_opts))
n_params = len(cv_params)
print(f'# Parameters: {n_params}')

# Parameters: 18


We randomly sample cv param combinations (here: 50%)

In [23]:
cvp = np.random.choice(list(range(n_params)),
                       size=int(n_params / 1),
                       replace=False)
cv_params_ = [cv_params[i] for i in cvp]

The number of iterations is another hyperparamter to optimize: To do so, we generate predictions for a range of iterations.

In [24]:
num_boost_rounds = [10, 25, 50, 75, 100, 150, 200] + list(range(250, 5001, 250))

### Run Cross-Validation

With this setup, we'll train 18 different models for 16 folds each, using the past five years to predict the next three months.

In [25]:
print(f'Train: {train_length:3.0f} | '
      f'Test: {test_length:2.0f} | '
      f'Folds: {n_splits:.0f} | '
      f'Params: {len(cv_params_):3.0f}')

Train: 1260 | Test: 63 | Folds: 16 | Params:  18


In [26]:
lgb_store = 'lgb_tuning.h5'

In [27]:
model_path = Path('models')
if not model_path.exists():
    model_path.mkdir()

Ready to go: for each fold, we'll plot the best iteration and the corresponding avg. daily IC and its coefficient of variation as well as the time per iteration.

In [ ]:
T = 0
for p, param_vals in enumerate(cv_params_):

    key = '/'.join([str(p) for p in param_vals])
    model_key = '_'.join(key.split('/'))
    params = dict(zip(param_names, param_vals))
    params.update(base_params)
    start = time()

    # create data structures to store results
    daily_ic_mean = np.full(shape=(len(num_boost_rounds),
                                   n_splits),
                            fill_value=np.nan)
    daily_ic_std = daily_ic_mean.copy()

    # train on data through 2016 to select the best model
    for n_split, (train_idx, test_idx) in enumerate(cv.split(X=data)):
        print(f'{n_split:2.0f} |', end=' ', flush=True)
        iteration = time()

        # select data points and create binary train set
        train_set = data.iloc[train_idx, :]
        lgb_train = lgb.Dataset(data=train_set.loc[:, features],
                                label=train_set[target],
                                categorical_feature=categoricals,
                                free_raw_data=False)

        # create test data
        test_set = data.iloc[test_idx, :]
        X_test = test_set.loc[:, features]
        y_test = test_set.loc[:, target].to_frame('y_true')
        
        # train model for max num_boost_rounds
        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=num_boost_rounds[-1],
                          verbose_eval=False)
        
        # persist model to generate out-of-sample predictions
        model_file = model_path / f'{model_key}_{n_split}.txt'
        model.save_model(model_file.as_posix())
        
        # evaluate trained model for various num_boost_rounds
        for i, num_boost_round in enumerate(num_boost_rounds):
            y_pred = model.predict(X_test, num_iteration=num_boost_round)
            y_test = y_test.assign(y_pred=y_pred)
            
            # compute daily information coefficient
            daily_ic = (y_test.groupby(level='date')
                        .apply(lambda x: spearmanr(x.y_true, x.y_pred)[0]))

            daily_ic_mean[i, n_split] = daily_ic.mean()
            daily_ic_std[i, n_split] = daily_ic.std()

        mu = pd.Series(daily_ic_mean[:, n_split], index=num_boost_rounds)
        s = pd.Series(daily_ic_std[:, n_split], index=num_boost_rounds)
        msg = f'{mu.idxmax():6,.0f} | {mu.max():6.2%} | {s[mu.idxmax()]/mu.max():6.1f} | '
        print(msg + f'{time() - iteration:5.1f}', flush=True)

    t = time() - start
    T += t
    
    ic = pd.DataFrame(daily_ic_mean, index=num_boost_rounds)
    ic_mean = ic.mean(1).max()
    ic_mean_n = ic.mean(1).idxmax()

    print(f'\n{p:3.0f} | {format_time(T)} | {format_time(t)} | {params["learning_rate"]:5.2f} | '
          f'{params["num_leaves"]:3.0f} | {params["feature_fraction"]:3.0%} | '
          f'{params["min_data_in_leaf"]:4.0f} | '
          f' {ic_mean:6.2%} | {ic_mean_n:5.0f}')
    
    ic.to_hdf(lgb_store, 'ic/' + key)

 0 |  4,500 |  1.83% |    8.4 | 103.8
 1 |     10 | -0.11% | -122.0 | 108.3
 2 |    750 |  1.21% |   17.6 | 145.0
 3 |  2,750 |  4.01% |    5.5 | 127.0
 4 |  2,500 |  3.78% |    3.8 | 145.6
 5 |    750 |  3.80% |    3.9 | 140.1
 6 |    500 |  1.70% |    7.6 | 143.1
 7 |  3,000 |  2.87% |    5.3 | 149.3
 8 |  2,500 |  3.14% |    6.1 | 138.7
 9 |    100 |  1.12% |    8.3 | 131.2
10 |  3,000 |  1.55% |    8.2 | 133.9
11 |  5,000 |  3.32% |    3.1 | 148.0
12 |  5,000 |  2.42% |    3.3 | 137.1
13 |    100 |  1.99% |    4.0 | 134.5
14 |     10 |  3.23% |    3.1 | 142.2
15 |  5,000 | -1.08% |   -9.1 | 159.3

  0 | 00:36:29 | 00:36:29 |  0.01 |  32 | 60% |  250 |   1.70% |  3000
 0 |  5,000 |  1.49% |    9.9 | 198.2
 1 |  5,000 | -0.42% |  -30.6 | 219.8
 2 |    250 |  2.08% |    9.4 | 206.7
 3 |  2,250 |  3.81% |    5.9 | 156.5
 4 |     25 |  4.21% |    3.1 | 145.1
 5 |  1,000 |  3.62% |    3.9 | 153.2
 6 |    750 |  2.08% |    5.9 | 144.8
 7 |  2,000 |  3.10% |    4.9 | 144.8
 8 |  3,250 |  3

### Collect Results

Here we gather the daily IC values for the various models alongside the matching hyperparameters.

In [ ]:
id_vars = ['num_iteration', 'learning_rate', 'num_leaves', 'feature_frac', 'num_data']

In [ ]:
results = []
with pd.HDFStore('lgb_tuning.h5') as store:
    for k in [k[1:] for k in store.keys() if k[1:].startswith('ic/')]:
        _, lr, num_leaves, feature_frac, num_data = k.split('/')
        results.append(store[k].assign(learning_rate=float(lr),
                                       num_leaves=int(num_leaves),
                                       feature_frac=float(feature_frac),
                                       num_data=int(num_data)))

In [ ]:
results = pd.concat(results).reset_index().rename(columns={'index': 'num_iteration'})
results = pd.melt(results,
                  id_vars=id_vars,
                  value_name='ic',
                  var_name='fold').apply(pd.to_numeric)

In [ ]:
results.info()

In [ ]:
results.head()

## Generate Predictions for best models

Now we'll select the parameters that produced the 10 best daily IC values over four consecutive quarters and generate predictions for these configurations for the following quarter, using the already trained model.

In [ ]:
param_names = id_vars[1:]

In [ ]:
predictions = []
for fold, (train_idx, test_idx) in enumerate(cv.split(X=data)):
    if fold >= 12:
        break
    print(fold, end=' ', flush=True)
    
    # get test features and outcomes
    test_data = data.iloc[test_idx]
    X_test = test_data.loc[:, features]
    fold_predictions = test_data.loc[:, target].to_frame('y_true')

    # select top 10 models based on prior 12 months performance
    best_params = (results[results.fold.isin(list(range(fold + 1, fold + 4)))]
                   .groupby(id_vars)
                   .ic.mean()
                   .nlargest(n=10).to_frame('ic'))
    
    # generate predictions for top 10 models
    for pos, params in enumerate(best_params.reset_index().to_dict('records')):
        num_iteration = params['num_iteration']
        param_vals = [params[p] for p in param_names]
        model_key = '_'.join([str(p) for p in param_vals])
        params = dict(zip(param_names, param_vals))
        params.update(base_params)

        # select matching model for current quarter
        model_path = Path('models/' + model_key + f'_{fold}.txt')
        model = lgb.Booster(model_file=model_path.as_posix())
        
        # predict returns based on current quarter features
        fold_predictions[pos] = model.predict(X_test.loc[:, model.feature_name()], 
                                              num_iteration=num_iteration)
    predictions.append(fold_predictions)
predictions = pd.concat(predictions)    

In [ ]:
predictions.sort_index().info()

In [ ]:
pred_cols = predictions.columns.drop('y_true')
for col in pred_cols:
    rho, p = spearmanr(predictions.y_true, predictions[col])
    print(f'Top {col}: {rho * 100:.2f} ({p:.2%})')

Let's compare the performance of individual models to that of an averaged ensemble:

In [ ]:
rho, p = spearmanr(predictions.y_true, predictions.loc[:, pred_cols].mean(1))
print(f'\nAvg: {rho * 100:.2f} ({p:.2%})')

We obtain an average IC of 3.01%.

In [ ]:
with pd.HDFStore('predictions.h5') as store:
    store.put(f'gradient_boosting', predictions.loc[:, pred_cols].mean(1))

## AlphaLens Analysis

Let's take a look how the model predictions perform from an alpha factor perspective.

### Prepare Factor Data

In [ ]:
t = 1
idx = pd.IndexSlice
factor = (predictions.loc[:, pred_cols].mean(1)
          .sort_index()
          .dropna()
          .tz_localize('UTC', level='date')
          .swaplevel())

In [ ]:
factor.head()

In [ ]:
dates = factor.index.unique('date')
dates.min(), dates.max()

In [ ]:
tickers = factor.index.unique('ticker')

### Select next available trade prices

Using next available prices.

In [ ]:
def get_trade_prices(tickers):
    return (pd.read_hdf('stock_prices.h5', 'us_stocks')
              .loc[idx[tickers, '2006':'2017'], 'open']
              .unstack('ticker')
              .sort_index()
            .shift(-1)
            .tz_localize('UTC'))

In [ ]:
trade_prices = get_trade_prices(tickers)
trade_prices.info()

### Get AlphaLens Inputs

In [ ]:
factor_data = get_clean_factor_and_forward_returns(factor=factor,
                                                   prices=trade_prices,
                                                   quantiles=10,
                                                   periods=(1, 2, 3, 5))
factor_data.sort_index().info()

### Summary Tearsheet

For a 1-day holding period, we find a meaningful (but asymmetric) spread between the top and bottom deciles; the results suggest that our model does best at identifying poor near-term performance:

In [ ]:
create_summary_tear_sheet(factor_data)